# Reverb CTM to Label Studio

> "Because their import sucks"

- hidden: true
- branch: master
- categories: [labelstudio, reverbasr, ctm, hsi]

In [2]:
def slurpfile(filename) -> str:
    with open(filename) as inf:
        return inf.read().strip()

In [3]:
host = "http://130.237.3.107:8080/api/"
api_token: str = slurpfile("label_studio_reverb")
input_dir = "/Users/joregan/Playing/hsi/inter_output/attention_rescoring/"

In [4]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [4]:
def get_projects():
    req = requests.get(f"{host}projects", headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [5]:
def get_project_id_from_name(name):
    projects = get_projects()
    for res in projects["results"]:
        if res["title"].strip() == name.strip():
            return res["id"]

In [9]:
get_project_id_from_name("Main 6")

7

In [10]:
def get_tasks(projectid):
    req = requests.get(f"{host}tasks", headers=headers, params={"project": projectid})
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [11]:
def index_task_filestem_to_id(tasks_data):
    tasks = tasks_data["tasks"]
    mapping = {}
    for task in tasks:
        task_id = task["id"]
        if "storage_filename" in task:
            task_raw_path = task["storage_filename"]
        else:
            task_raw_path = task["data"]["audio"]
        if not task_raw_path:
            continue
        task_stem = task_raw_path.split("/")[-1]
        mapping[task_stem] = task_id
    return mapping

In [22]:
get_project_id_from_name("Speaker 3")

1

In [43]:
tasks = get_tasks(8)

In [44]:
mapping = index_task_filestem_to_id(tasks)

In [45]:
mapping

{'hsi_7_0719_209_001_main.wav': 107,
 'hsi_7_0719_209_002_main.wav': 108,
 'hsi_7_0719_209_003_main.wav': 109,
 'hsi_7_0719_210_001_main.wav': 110,
 'hsi_7_0719_210_002_main.wav': 111,
 'hsi_7_0719_210_003_main.wav': 112,
 'hsi_7_0719_211_002_main.wav': 113,
 'hsi_7_0719_211_004_main.wav': 114,
 'hsi_7_0719_222_002_main.wav': 115,
 'hsi_7_0719_222_004_main.wav': 116,
 'hsi_7_0719_227_002_main.wav': 117,
 'hsi_7_0719_227_003_main.wav': 118}

In [12]:
ctmfile = "/Users/joregan/Playing/hsi/inter_output/attention_rescoring/hsi_7_0719_209_002_inter.ctm"
outputs = []

segments = []

with open(ctmfile) as ctmf:
    def is_marker(item):
        if item["word"].startswith("<") and item["word"].endswith(">"):
            item["word"] = f'[{item["word"][1:-1]}]'
            return True
        return False

    last = 0.0
    this_seg = []
    for line in ctmf.readlines():
        line = line.strip()
        parts = line.split(" ")
        cur = {
            "start": float(parts[2]),
            "dur": float(parts[3]),
            "word": parts[4],
            "end": float(parts[2]) + float(parts[3]),
        }
        if (cur["end"] - last) > 1.0 or is_marker(cur):
            segments.append(this_seg)
            this_seg = []
        this_seg.append(cur)
        last = cur["end"]
    segments.append(this_seg)

In [ ]:
segments

In [39]:
import json
import uuid


def ctm_to_result(ctmfile):
    def is_marker(item):
        if item["word"].startswith("<") and item["word"].endswith(">"):
            item["word"] = f'[{item["word"][1:-1]}]'
            return True
        return False

    outputs = []

    segments = []

    with open(ctmfile) as ctmf:
        last = 0.0
        this_seg = []
        for line in ctmf.readlines():
            line = line.strip()
            parts = line.split(" ")
            cur = {
                "start": float(parts[2]),
                "dur": float(parts[3]),
                "word": parts[4],
                "end": float(parts[2]) + float(parts[3]),
            }
            if (cur["end"] - last) > 1.0 or is_marker(cur):
                segments.append(this_seg)
                this_seg = []
            this_seg.append(cur)
            last = cur["end"]
        segments.append(this_seg)

    for segment in segments:
        if segment == []:
            continue

        start = segment[0]["start"]
        end = segment[-1]["end"]
        text = " ".join([x["word"] for x in segment])

        add_label = False
        label = "Speech"
        if text == "[laugh]":
            add_label = True
            label = "Laughter"
        elif text == "[inaudible]":
            add_label = True
            label = "Noise"

        gen_id = str(uuid.uuid4())[:6]
        segment = {
            "value": {
                "start": start,
                "end": end,
                "channel": 0,
                "labels": [label]
            },
            "from_name": "labels",
            "to_name": "audio",
            "type": "labels",
            "id": gen_id
        }
        rec = {
            "value": {
                "start": start,
                "end": end,
                "channel": 0,
                "text": [text]
            },
            "from_name": "transcription",
            "to_name": "audio",
            "type": "textarea",
            "id": gen_id
        }
        if add_label:
            outputs.append(segment)
        outputs.append(rec)

    return outputs

In [15]:
def post_results(id, task, project, results):
    ep = f"{host}annotations/{id}/?taskID={task}&project={project}"

    cur_headers = {i: headers[i] for i in headers}
    cur_headers["Content-type"] = "application/json"

    content = {
        "was_cancelled": False,
        "ground_truth": False,
        "project": project,
        "draft_id": 0,
        "parent_prediction": None,
        "parent_annotation": None,
        "result": results
    }
    r = requests.patch(ep, data=json.dumps(content), headers=cur_headers)
    return r

In [40]:
file = f"{input_dir}hsi_7_0719_209_002_inter.ctm"
data = ctm_to_result(file)

In [42]:
r = post_results(300, 55, 4, data)
print(r.text)

{"id":300,"result":[{"value":{"start":1.95,"end":4.6899999999999995,"channel":0,"text":["yeah but just let me know because it is an easy fix to"]},"from_name":"transcription","to_name":"audio","type":"textarea","id":"35e926"},{"value":{"start":6.15,"end":6.25,"channel":0,"text":["um"]},"from_name":"transcription","to_name":"audio","type":"textarea","id":"537955"},{"value":{"start":7.47,"end":12.049999999999999,"channel":0,"text":["but yeah i think it looks good i started it now so do you need to uh point that"]},"from_name":"transcription","to_name":"audio","type":"textarea","id":"d363c0"},{"value":{"start":12.47,"end":17.85,"channel":0,"text":["[inaudible] uh yeah i think that's good if you do that but maybe in this one because it's a landlord so maybe you can be very picky about stuff okay"]},"from_name":"transcription","to_name":"audio","type":"textarea","id":"5e6cdd"},{"value":{"start":18.79,"end":35.160000000000004,"channel":0,"text":["um but yeah was very nice living here for fou

In [ ]:
from pathlib import Path

for fn in mapping:
    ctmfile = fn.replace(".wav", ".TextGrid")
    if (Path(input_dir) / ctmfile).exists():
        print(mapping[fn], fn)


In [ ]:
count = 230
for task in mapping:
    jsonfile = task.replace(".wav", ".TextGrid")
    file = f"{input_dir}{jsonfile}"
    if not (Path(input_dir) / jsonfile).exists():
        continue
    data = tg_to_result(file)
    r = post_results(count, mapping[task], 8, data)
    count += 1
    print(r.text)

In [136]:
tmap = {}
count = 99
for task in mapping:
    tmap[task] = count
    count += 1


In [ ]:
from pathlib import Path

for file in Path("/Users/joregan/Playing/hsi_ctmedit/textgrid").glob("*.TextGrid"):
    wavfile = file.stem + ".wav"
    if wavfile in mapping:
        print(wavfile, mapping[wavfile])